In [11]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
root_dir = '..'
sys.path.append(root_dir)
import utils
from LinearAlignment import LinearAlignment

In [5]:
data = pd.read_csv(os.path.join(root_dir, 'data/train.csv'))
data['epoch'] = pd.to_datetime(data['epoch']).astype(int)

In [6]:
features_list=('x', 'y', 'z', 'Vx', 'Vy', 'Vz')

In [25]:
a = pd.DataFrame(np.ones((4, 6)))
b = pd.DataFrame(np.ones((10, 7)))

In [26]:
pd.concat([a, b])

,0,1,2,3,4,5,6
0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,NaN
3,1.0,1.0,1.0,1.0,1.0,1.0,NaN
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
pd.concat(a, b)

,0,1,2,3,4
0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0


In [7]:
result_df = []
alignment_model = LinearAlignment()

satellites_list = data['sat_id'].unique()
for sat_id in tqdm(satellites_list):
    pred = pd.DataFrame([])
    sat_data = utils.get_satellite_data(data, sat_id)
    try:
        sat_data = utils.remove_time_jumps_fast(sat_data)
#         filtered_sat_data = utils.remove_time_jumps(sat_data)
    except KeyError as e:
        print(f'jump removal failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
#     sat_data = sat_data.join(utils.remove_time_jumps(sat_data))
    n_train = len(sat_data) // 2
    train_sat_data = sat_data[:n_train]
    
    try:
        for feature_name in features_list:
            alignment_model.fit(t=train_sat_data['epoch'].values,
                                x=train_sat_data[f'{feature_name}_sim'].values,
                                gt=train_sat_data[feature_name].values)
            pred[feature_name] = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
    except Exception as e:
        print(f'linear alignment failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
        
    sat_sim_smape = utils.smape(sat_data.loc[n_train:, [f'{f}_sim' for f in features_list]].values,
                                sat_data.loc[n_train:, features_list].values)
    sat_new_smape = utils.smape(pred.loc[n_train:, features_list].values,
                                sat_data.loc[n_train:, features_list].values)
    result_df.append([sat_id, sat_sim_smape, sat_new_smape])
        

  0%|          | 0/600 [00:00<?, ?it/s]/home/misha/python-virtual-environments/py373-venv/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
 42%|████▏     | 251/600 [00:19<00:27, 12.67it/s]/home/misha/Documents/Competitions/IDAO-2020/LinearAlignment/LinearAlignment.py:39: RuntimeWarning: invalid value encountered in long_scalars
../utils.py:20: RuntimeWarning: invalid value encountered in greater
  far = np.concatenate([(t[1:] - t[:-1]) > eps, [True]])
 42%|████▎     | 255/600 [00:19<00:25, 13.61it/s]

linear alignment failed for satellite 252:	ValueError interpolation error, x length = 1,         t_new length = 94


100%|██████████| 600/600 [00:46<00:00, 12.82it/s]


In [6]:
result_df = pd.DataFrame(result_df, columns=['sat_id', 'sat_simulation_smape', 'sat_new_smape'])

In [7]:
print(f'{len(result_df)}/{len(satellites_list)} satellites processed')
print('\nmetrics:', dict(result_df.iloc[:, 1:].mean()))

599/600 satellites processed

metrics: {'sat_simulation_smape': 0.14499453179384694, 'sat_new_smape': 0.08145263918265075}


In [9]:
print(f'{len(result_df)}/{len(satellites_list)} satellites processed')
print('\nmetrics:', dict(result_df.iloc[:, 1:].mean()))

587/600 satellites processed

metrics: {'sat_simulation_smape': 0.2132816584458697, 'sat_new_smape': 0.12919047242986784}
